In [6]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ambon,ID,-3.6954,128.1814,78.94,89,20,4.61,few clouds
1,1,Komsomolskiy,UZ,40.4272,71.7189,44.02,69,15,1.30,few clouds
2,2,Bredasdorp,ZA,-34.5322,20.0403,59.29,80,82,2.91,broken clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,70.56,87,60,5.01,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.80,80,98,18.25,overcast clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ambon,ID,-3.6954,128.1814,78.94,89,20,4.61,few clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,70.56,87,60,5.01,broken clouds
4,4,Rikitea,PF,-23.1203,-134.9692,74.80,80,98,18.25,overcast clouds
6,6,Georgetown,MY,5.4112,100.3354,78.89,86,20,3.44,few clouds
8,8,Bowen,AU,-20.0167,148.2333,77.50,57,9,8.88,clear sky
10,10,Tual,ID,-5.6667,132.7500,82.58,74,100,1.63,overcast clouds
17,17,Wanning,CN,18.8003,110.3967,79.05,88,97,3.33,overcast clouds
22,22,Faanui,PF,-16.4833,-151.7500,79.66,74,48,12.19,light rain
26,26,Vaini,TO,-21.2000,-175.2000,78.96,83,75,6.91,broken clouds
27,27,Kieta,PG,-6.2167,155.6333,83.88,69,85,5.91,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ambon,ID,78.94,few clouds,-3.6954,128.1814,
3,Puerto Ayora,EC,70.56,broken clouds,-0.7393,-90.3518,
4,Rikitea,PF,74.80,overcast clouds,-23.1203,-134.9692,
6,Georgetown,MY,78.89,few clouds,5.4112,100.3354,
8,Bowen,AU,77.50,clear sky,-20.0167,148.2333,
10,Tual,ID,82.58,overcast clouds,-5.6667,132.7500,
17,Wanning,CN,79.05,overcast clouds,18.8003,110.3967,
22,Faanui,PF,79.66,light rain,-16.4833,-151.7500,
26,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
27,Kieta,PG,83.88,overcast clouds,-6.2167,155.6333,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ambon,ID,78.94,few clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
3,Puerto Ayora,EC,70.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
4,Rikitea,PF,74.80,overcast clouds,-23.1203,-134.9692,People ThankYou
6,Georgetown,MY,78.89,few clouds,5.4112,100.3354,Cititel Penang
8,Bowen,AU,77.50,clear sky,-20.0167,148.2333,Castle Motor Lodge Motel


In [16]:

# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))